# Implement seq2seq
---

## 1. 필요한 모듈을 불러온다.

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from pprint import pprint # print() 함수의 결과를 아름답게(beautify) 보여주는 모듈
from tools import * # 직접만든 함수 모음.
from tqdm import tqdm # 배치학습 for문을 예쁘게 시각화 하는 모듈
from tqdm import trange
import math

tf.reset_default_graph() # 텐서플로우 그래프를 초기화 한다.

## 2. corpus(YOLO의 output)를 불러온다.

In [3]:
# pandas DataFrame을 numpy.ndarray로 바꾸려면 '.values'만 추가하면 된다.
# => pd.read_table(...).values
corpus = pd.read_table("./raw.csv", delimiter=",").values

print((corpus, type(corpus)))
print("\ncorpus's shape: {}".format(corpus.shape))

(array([['dallas take off', ' plane take off from dallas'],
       ['dallas land', ' plane land at Dallas'],
       ['chicago take off', ' plane take off from chicago'],
       ['chicago land', ' plane land at chicago'],
       ['los angeles take off', ' plane take off from los angeles'],
       ['los angeles land', ' plane land at los angeles'],
       ['new york take off', ' plane take off from new york'],
       ['new york land', ' plane land at new york'],
       ['miami take off', ' plane take off from miami'],
       ['miami land', ' plane land at miami']], dtype=object), <class 'numpy.ndarray'>)

corpus's shape: (10, 2)


## 3. lookup table를 생성하고 이를 이용하여 데이터를 인덱스로 맵핑한다.

- seq2seq 모델의 cell은 인풋값으로 index로 된 numpy array나 리스트를 받는다.
- tools.make_vocab(numpy.ndarray)
    - numpy.ndarray: [batch size x time steps]

In [4]:
# Make idx2word, word2idx, vocab_size
idx2word, word2idx, vocab_size = make_vocab(corpus)

# make_temp_data(numpy.ndarray, idx2word, word2idx)
# => corpus를 인덱스 값으로 변환 시킨후 return한다. (<pad>, <start>, <end>, <unk> 토큰은 삽입되지 않음)
# Dimension: batch size X time steps
temp_encoder_data, temp_decoder_data, temp_targets_data = make_temp_data(corpus, idx2word=idx2word, word2idx=word2idx)

# Identify temp_encoder_data, temp_decoder_data, temp_targets_data
print('vocab_size: {} \n'.format(vocab_size))
print('idx2word: \n {} \n'.format(idx2word))
print('word2idx: \n {} \n'.format(word2idx))

# 패딩 토큰이 삽입되지 않아서 배치마다 time steps의 길이가 다르다. 이후에 수정 해 줄 것이다.
print('temp_encoder_data: \n {} \n'.format(temp_encoder_data))
print('temp_decoder_data: \n {} \n'.format(temp_decoder_data))
print('temp_targetsr_data: \n {} \n'.format(temp_targets_data))

vocab_size: 17 

idx2word: 
 {0: 'land', 1: 'at', 2: 'plane', 3: 'york', 4: 'angeles', 5: 'los', 6: 'dallas', 7: 'chicago', 8: 'new', 9: 'off', 10: 'take', 11: 'from', 12: 'miami', 13: '<start>', 14: '<end>', 15: '<pad>', 16: '<unk>'} 

word2idx: 
 {'land': 0, 'at': 1, 'plane': 2, 'york': 3, 'angeles': 4, 'los': 5, 'dallas': 6, 'chicago': 7, 'new': 8, 'off': 9, 'take': 10, 'from': 11, 'miami': 12, '<start>': 13, '<end>': 14, '<pad>': 15, '<unk>': 16} 

temp_encoder_data: 
 [[6, 10, 9], [6, 0], [7, 10, 9], [7, 0], [5, 4, 10, 9], [5, 4, 0], [8, 3, 10, 9], [8, 3, 0], [12, 10, 9], [12, 0]] 

temp_decoder_data: 
 [[2, 10, 9, 11, 6], [2, 0, 1, 6], [2, 10, 9, 11, 7], [2, 0, 1, 7], [2, 10, 9, 11, 5, 4], [2, 0, 1, 5, 4], [2, 10, 9, 11, 8, 3], [2, 0, 1, 8, 3], [2, 10, 9, 11, 12], [2, 0, 1, 12]] 

temp_targetsr_data: 
 [[2, 10, 9, 11, 6], [2, 0, 1, 6], [2, 10, 9, 11, 7], [2, 0, 1, 7], [2, 10, 9, 11, 5, 4], [2, 0, 1, 5, 4], [2, 10, 9, 11, 8, 3], [2, 0, 1, 8, 3], [2, 10, 9, 11, 12], [2, 0, 1, 12]] 

## 4. 각 데이터에 pad, end 토큰을 삽입하여 최종적인 데이터를 만든다.

In [5]:
# tools.insert_tokens(numpy.ndarray, word2idx)
# => insert_tokens()의 argument numpy.ndarray는 index로 변환 된 데이터를 받는다. 주의하자.
# Dimension: batch size X time step
encoder_data, decoder_data, targets_data, max_decoder_steps = insert_tokens(temp_encoder_data, temp_decoder_data, temp_targets_data, word2idx)

# 최종적인 데이터가 잘 생성 되었는지를 확인하려면 아래 코드를 실행해보자.(옵션)
# data_checker(encoder_data, idx2word)
# data_checker(decoder_data, idx2word)
# data_checker(targets_data, idx2word)

## 5. mask를 사용하기 위해서 sequence_length를 구한다.

In [6]:
# 각 배치에서 <pad>를 제외한 실제 의미 있는 time steps의 길이를 구한다.
# Dimension: batch size(1-D)
encoder_sequence_length = sequence_length_maker(encoder_data, word2idx)
decoder_sequence_length = sequence_length_maker(decoder_data, word2idx)

print(encoder_sequence_length)
print(decoder_sequence_length)

[3, 2, 3, 2, 4, 3, 4, 3, 3, 2]
[6, 5, 6, 5, 7, 6, 7, 6, 6, 5]


## 6. 하이퍼파라미터를 정의한다.

In [7]:
# decoder의 outputs을 projection layer에 통과시키면 batch size X time step X class_size의 dimension을 얻게 된다.
# 원-핫 인코딩이면 vocab_size와 같게 해 주어야 한다.
class_size = vocab_size
batch_size = 3
num_batch = math.ceil(encoder_data.shape[0]/batch_size)
lr = 0.01
# seq2seq셀에 흐르는 hidden state의 유닛개수
hidden_size = 128
epochs = 100
save_per_ckpt = 10
# stacked RNN을 위한 하이퍼파라미터. 몇개의 레이어를 쌓을지 정해준다.
num_layer = 1 

## 7. 모델을 클래스로 정의한다.

In [8]:
class Object2Text:
    # 'btc_': 배치
    '''
    Arguments:
        btc_enc_seq_len: placeholder([None,]), 인코더 부분의 패딩을 제외한 실제 의미를 가지는 토큰의 개수를 배치별로 나타냄.
        btc_enc_inp: placeholder([None, None]), 인코더의 인풋(batch size, time steps)
        btc_dec_seq_len: placeholder([None,]), 디코더 부분의 패딩을 제외한 실제 의미를 가지는 토큰의 개수를 배치별로 나타냄.
        btc_dec_inp: placeholder([Noen, None]), 디코더의 인풋(batch size, time steps)
        btc_targets: placeholder([None, None]), groud truth(batch size, time steps)
        word2idx: dictionary, '단어'를 키 값으로 조회하면 그에 해당하는 인덱스를 돌려주는 딕셔너리.
        idx2word: dictionary, '인덱스'를 키 값으로 조회하면 그에 해당하는 인덱스를 돌려주는 딕셔너리.
        vocab_size: scalar, 단어사전의 크기(단어사전에 있는 단어의 개수)
        class_size: scalar, 디코더의 아웃풋을 fully connected 레이어와 연결하여 최종적으로 몇개의 클래스로 나타낼지 결정. 일반적으로 vocab size와 같게 하면 된다.
        hidden_size: scalar, seq2seq 모델의 히든 유닛의 개수
    '''
    
    def __init__(self, btc_enc_seq_len, btc_enc_inp, btc_dec_seq_len, btc_dec_inp, btc_targets, word2idx=word2idx, idx2word=idx2word, 
                     vocab_size=vocab_size, class_size=class_size, hidden_size=hidden_size, num_layer=num_layer, max_decoder_steps=max_decoder_steps):
        
        # 생성자의 인자들을 클래스멤버(?)로 만들어준다.
        self._enc_seq_len = btc_enc_seq_len
        self._enc_inp = btc_enc_inp
        self._dec_seq_len = btc_dec_seq_len
        self._dec_inp = btc_dec_inp
        self._targets = btc_targets
        self._word2idx = word2idx
        self._idx2word = idx2word
        self._vocab_size = vocab_size
        self._class_size = class_size
        self._hidden_size = hidden_size
        self._num_layer = num_layer
        self._max_decoder_steps = max_decoder_steps
    
        # 1) encoder의 인풋값을 ont-hot 인코딩(임베딩) 한다.
        with tf.variable_scope('enc_embed_layer'):
            encoder_oh = tf.one_hot(indices=self._enc_inp, depth=self._vocab_size) # encoder_oh: tensor, (batch_size, time steps, vocab_size)

        #  2) encoder 셀을 쌓는다.
        with tf.variable_scope('encoder'):
            enc_cell = tf.contrib.rnn.BasicLSTMCell(num_units=self._hidden_size, activation=tf.nn.tanh)
            enc_cell = tf.nn.rnn_cell.MultiRNNCell([enc_cell] * self._num_layer)
            _outputs, enc_states = tf.nn.dynamic_rnn(cell=enc_cell, inputs=encoder_oh, sequence_length=self._enc_seq_len, dtype=tf.float32)

        # 3) 디코더의 임베딩을 원-핫인코딩으로 한다. tf.one_hot 함수를 사용해서 원-핫 인코딩을 해도 되지만. 나중에 GreedyEmbeddingsHelper를 사용하기 위해서는 임베딩 매트릭스를 만들어 주어야 한다.
        with tf.variable_scope('dec_embed_layer'):
            decoder_eye = tf.eye(num_rows=self._vocab_size)
            decoder_embeddings = tf.get_variable(name='dec_embed', initializer=decoder_eye, trainable=False) # tf.nn.embedding_lookup 함수를 사용하려면 tf.get_variable을 params의 값으로

            #decoder_embeddings = tf.get_variable(name='dec_embed', initializer=decoder_eye, trainable=False) # tf.nn.embedding_lookup 함수를 사용하려면 tf.get_variable을 params의 값으로
            decoder_oh = tf.nn.embedding_lookup(params=decoder_embeddings, ids=self._dec_inp)    

        # 4) decoder 셀을 쌓는다.    
        with tf.variable_scope('decoder'):
            
            # decoder cell for both training and inference(학습과 추론과정은 같은 '가중치'를 공유함.)
            dec_cell = tf.contrib.rnn.BasicLSTMCell(self._hidden_size, activation=tf.nn.tanh)
            dec_cell = tf.nn.rnn_cell.MultiRNNCell([dec_cell] * self._num_layer)
            
            # output projection (replacing `OutputProjectionWrapper`)
            output_layer = tf.layers.Dense(vocab_size)
#             score_cell = tf.contrib.rnn.OutputProjectionWrapper(cell=dec_cell, output_size=self._class_size)

            # 4-1) decoder의 training과 관련된 셀을 쌓는다.
            with tf.variable_scope('training'):
                train_helper = tf.contrib.seq2seq.TrainingHelper(inputs=decoder_oh, sequence_length=self._dec_seq_len, time_major=False)
                train_decoder = tf.contrib.seq2seq.BasicDecoder(cell=dec_cell, helper=train_helper, initial_state=enc_states, output_layer=output_layer)
                
                # self._train_outputs.rnn_output: [batch_size x max(self._max_decoder_steps) x vocab_size]
                # => 각 배치에서 가장 '긴' 값을 2번째 차원의 값으로 가진다는 점을 주의!! seq2seq2 loss 함수를 사용할 때 shape에 주의해야 한다.
                # => bucketing과 연관이 있는 것 같은데.... 확실하지는 않다...
                self._train_outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder=train_decoder, impute_finished=True, output_time_major=False, 
                                                                                                                maximum_iterations=self._max_decoder_steps)
                
                # logits: [batch_size x max_dec_step x dec_vocab_size]
                # => tf.identity: seq2seq 함수에서 logits에 들어갈 값 선언. tf.identity는 tensor의 contents와 shape를 똑같이 copy한다.
                logits = tf.identity(self._train_outputs.rnn_output, name='logits')
                
                # btc_max_len: => logits의 shape에 맞춰주기 위함!!
                # => decoder input "배치" 중에서  최대 sequence length(의미 있는 인풋이 들어오는 time step)
                btc_max_len = tf.reduce_max(self._dec_seq_len, name='max_dec_len')
                
                # targets: 
                targets = tf.slice(input_=self._targets, begin=[0, 0], size=[-1, btc_max_len], name='targets')
                              

            # 4-2) decoder의 inference와 관련된 셀을 쌓는다.
            with tf.variable_scope('inference'):
                # 모델의 input을 dynamic shape로 하였으므로.. tf.shape(btc_enc_seq_len)로 해야 한다. btc_enc_seq_len.shape는 static shape일 때 사용 하는 것이다.
                start_tokens = tf.fill(dims=tf.shape(btc_enc_seq_len), value=self._word2idx['<start>'])

                infer_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(embedding=decoder_embeddings, 
                                                                                                              start_tokens=start_tokens, 
                                                                                                              end_token=self._word2idx['<end>'])
                infer_decoder = tf.contrib.seq2seq.BasicDecoder(cell=dec_cell, helper=infer_helper, initial_state=enc_states, output_layer=output_layer)
                self._infer_outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(infer_decoder, impute_finished=True, maximum_iterations=self._max_decoder_steps)
        
        with tf.variable_scope('seq2seq_loss'):
            #masksing: [batch_size x max_dec_len]
            #=> ignore outputs after `dec_senquence_length+1` when calculating loss
            masking = tf.sequence_mask(lengths=self._dec_seq_len, maxlen=btc_max_len, dtype=tf.float32)
                
            #internal: `tf.nn.sparse_softmax_cross_entropy_with_logits`
            self.seq2seq_loss = tf.contrib.seq2seq.sequence_loss(logits=logits, targets=targets, weights=masking)
            
    def infer(self, sess,  btc_enc_seq_len, btc_enc_inp):
        feed_infer = {self._enc_seq_len: btc_enc_seq_len, self._enc_inp: btc_enc_inp}
        return sess.run(self._infer_outputs.sample_id, feed_dict=feed_infer)            

## 8. 배치 관련 설정을 한다.

In [9]:
## 인풋, 타겟, sequence_length와 관련된 placeholder를 정의한다.
encoder_input = tf.placeholder(dtype=tf.int32, shape=[None, None]) # (batch size, time steps)
decoder_input = tf.placeholder(tf.int32, [None, None]) # (batch size, time steps)
targets = tf.placeholder(tf.int32, [None, None]) # (batch size, time steps)
encoder_seq_len = tf.placeholder(tf.int32, [None]) # (batch size, ) 인코더 배치에 대응하는 sequence_length
decoder_seq_len = tf.placeholder(tf.int32, [None]) # (batch size,) 디코더 배치에 대응하는 sequence_length

## 배치 iterator를 정의하여 학습시 사용할 준비를 한다.
dataset = tf.data.Dataset.from_tensor_slices((encoder_seq_len, encoder_input, decoder_seq_len, decoder_input, targets)) # 여러 플레이스 홀더를 데이터셋으로 하고
dataset = dataset.shuffle(10000).batch(batch_size) # 랜덤으로 데이터셋을 섞은 다음에 배치크기는 미리 정의된 만큼 할 생각이야.
iterator = dataset.make_initializable_iterator() # 위와 같은 사항을 반영하여 전체 데이터셋에서 배치 크기만큼 반복적으로 꺼내올 수 있는 iterator를 만들어 줘.
btc_enc_seq_len, btc_enc_inp, btc_dec_seq_len, btc_dec_inp, btc_targets  = iterator.get_next() # 배치를 꺼내오는 명령문은 next_batch로 할게. (sess.run에서 실행 시켜야 다음 배치를 꺼내온다.)
# next_element = iterator.get_next() # 배치를 꺼내오는 명령문은 next_batch로 할게. (sess.run에서 실행 시켜야 다음 배치를 꺼내온다.)

print(dataset)
# print(next_element)
print(btc_enc_seq_len, btc_enc_inp, btc_dec_seq_len, btc_dec_inp, btc_targets)

<BatchDataset shapes: ((?,), (?, ?), (?,), (?, ?), (?, ?)), types: (tf.int32, tf.int32, tf.int32, tf.int32, tf.int32)>
Tensor("IteratorGetNext:0", shape=(?,), dtype=int32) Tensor("IteratorGetNext:1", shape=(?, ?), dtype=int32) Tensor("IteratorGetNext:2", shape=(?,), dtype=int32) Tensor("IteratorGetNext:3", shape=(?, ?), dtype=int32) Tensor("IteratorGetNext:4", shape=(?, ?), dtype=int32)


## 9. object2text 클래스를 생성한다.

In [11]:
object2text = Object2Text(btc_enc_seq_len, btc_enc_inp, btc_dec_seq_len, btc_dec_inp, btc_targets)

AttributeError: module 'dask.dataframe' has no attribute 'Series'

In [13]:
import dask.dataframe as dd
dd.Series

AttributeError: module 'dask.dataframe' has no attribute 'Series'

## 10. optimizer를 정의하고 모델을 학습시킨다.

In [ ]:
train_op = tf.train.AdamOptimizer(lr).minimize(object2text.seq2seq_loss)

In [11]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
loss_hist=[]

for i in trange(epochs, desc='Epoch Progress: '):

    # iterator.initalizer를 반복문 안에 넣어줘야 배치를 다 꺼내 쓴 다음에 다시 배치를 쓸 수 있다. 이것을 반복문 바깥에 두면 계속 OutOfRangeError 에러가 난다.
    sess.run(iterator.initializer, feed_dict={encoder_input: encoder_data, decoder_input: decoder_data,
                                              targets: targets_data, encoder_seq_len: encoder_sequence_length, 
                                              decoder_seq_len: decoder_sequence_length})
    
    for j in trange(num_batch, desc="Batch Proress"):
        try:
            _, loss = sess.run([train_op, object2text.seq2seq_loss])

        # 에러가 발생한다는 것은 더 이상 꺼내 올 배치가 없다는 것이다. 따라서 배치를 꺼내오는 while문을 종료하고 다음 epoch로 넘어간다.
        except tf.errors.OutOfRangeError:
            break

# from tqdm import trange
# from time import sleep

# for i in trange(10, desc='1st loop'):
#     for j in trange(5, desc='2nd loop', leave=False):
#         sleep(0.01)

# t = trange(100, desc='seq2seq traing: ', leave=True, bar_format='{l_bar}{bar}{r_bar}')
# for epoch in t:

#     # iterator.initalizer를 반복문 안에 넣어줘야 배치를 다 꺼내 쓴 다음에 다시 배치를 쓸 수 있다. 이것을 반복문 바깥에 두면 계속 OutOfRangeError 에러가 난다.
#     sess.run(iterator.initializer, feed_dict={encoder_input: encoder_data, decoder_input: decoder_data,
#                                               targets: targets_data, encoder_seq_len: encoder_sequence_length, 
#                                               decoder_seq_len: decoder_sequence_length})

#     while True:
#         try:
#             _, loss = sess.run([train_op, object2text.seq2seq_loss])

#         # 에러가 발생한다는 것은 더 이상 꺼내 올 배치가 없다는 것이다. 따라서 배치를 꺼내오는 while문을 종료하고 다음 epoch로 넘어간다.
#         except tf.errors.OutOfRangeError:
#             break
    
#     # Epoch당 loss를 나타내주는 tqdm 함수.
#     t.set_postfix({"Epoch loss": loss})




Epoch Progress: 100%|██████████| 100/100 [00:03<00:00, 32.34it/s]


## 11. 학습된 가중치가 저장된 sess를 통해서 inference를 한다.

In [12]:
prediction = object2text.infer(sess, btc_enc_inp=encoder_data, btc_enc_seq_len=encoder_sequence_length)

In [13]:
print(prediction)

[[ 3  9  1  5 12 14  0]
 [ 3  8  0 12 14  0  0]
 [ 3  9  1  5  2 14  0]
 [ 3  8  0  2 14  0  0]
 [ 3  9  1  5  4  6 14]
 [ 3  8  0  4  6 14  0]
 [ 3  9  1  5 11  7 14]
 [ 3  8  0 11  7 14  0]
 [ 3  9  1  5 10 14  0]
 [ 3  8  0 10 14  0  0]]


In [14]:
list(map(lambda row: [idx2word[idx] for idx in row], prediction))

[['plane', 'take', 'off', 'from', 'dallas', '<end>', 'at'],
 ['plane', 'land', 'at', 'dallas', '<end>', 'at', 'at'],
 ['plane', 'take', 'off', 'from', 'chicago', '<end>', 'at'],
 ['plane', 'land', 'at', 'chicago', '<end>', 'at', 'at'],
 ['plane', 'take', 'off', 'from', 'los', 'angeles', '<end>'],
 ['plane', 'land', 'at', 'los', 'angeles', '<end>', 'at'],
 ['plane', 'take', 'off', 'from', 'new', 'york', '<end>'],
 ['plane', 'land', 'at', 'new', 'york', '<end>', 'at'],
 ['plane', 'take', 'off', 'from', 'miami', '<end>', 'at'],
 ['plane', 'land', 'at', 'miami', '<end>', 'at', 'at']]

In [ ]:
from tqdm import tqdm
from tqdm import tqdm_notebook
import time
import random

epochs = 10

for epoch in range(epochs):
    avg_loss = 0
    num_batch = 50
    
    pbar = tqdm_notebook(range(num_batch))
    
    for i in pbar:
        avg_loss += random.randint(1,10)
        pbar.set_description("Epoch {}/{} ".format(epoch+1, epochs))
        time.sleep(0.1)
    
    print(avg_loss)